In [ ]:
import requests
from weasyprint import HTML, CSS
from pathlib import Path
import os

BASE_URL = "https://chilamboli.devmorphix.com"
ASSETS_DIR = "assets"
OUTPUT_DIR = "output"
TEMPLATE_PATH = os.path.join(ASSETS_DIR, "report_template.html")
FONT_PATH = os.path.join(ASSETS_DIR, "Chilanka-Regular.ttf")

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"Config loaded: BASE_URL={BASE_URL}")
print(f"Assets directory: {ASSETS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Template: {TEMPLATE_PATH}")

In [ ]:
def fetch_event_results(event_id):
    """Fetch event results data from API"""
    url = f"{BASE_URL}/api/admin/events/{event_id}/results-data"
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    data = response.json()
    
    if not data.get("success"):
        raise RuntimeError(data.get("message", "API error"))
    
    return data

def load_template():
    """Load HTML template"""
    if not os.path.exists(TEMPLATE_PATH):
        raise FileNotFoundError(f"Template file not found: {TEMPLATE_PATH}")
    with open(TEMPLATE_PATH, 'r', encoding='utf-8') as f:
        return f.read()

print("Helper functions defined: fetch_event_results, load_template")

In [ ]:
def escape_html(text):
    """Escape HTML special characters"""
    if text is None:
        return ""
    text = str(text)
    return (text.replace('&', '&amp;')
                .replace('<', '&lt;')
                .replace('>', '&gt;')
                .replace('"', '&quot;')
                .replace("'", '&#x27;'))

def create_report_pdf(event_data, output_filename):
    """Create a PDF report from HTML template"""
    
    # Extract data
    event = event_data["event"]
    judges = event_data["judges"]
    results = event_data["results"]
    
    # Load template
    template = load_template()
    
    num_judges = len(judges)
    # Total columns: Rank(1) + Chest(1) + Team(1) + School(1) + Judges + Total(1) + GradeScore(1) + Grade(1) + Points(1)
    total_columns = 4 + num_judges + 4
    
    # Calculate column spans for event info row
    # Event spans 3 columns (Rank, Chest, Team)
    # School (1) + Judges (num_judges) are kept free
    # Type and Category each get 2 columns from the last 4 columns
    event_info_colspan = 3
    empty_span = 1 + num_judges  # School + Judges columns
    type_info_colspan = 2
    category_info_colspan = 2
    
    # Build judge headers for table
    judge_headers = ""
    for i in range(num_judges):
        judge_headers += f'<th class="col-judge">Judge-{i+1}</th>'
    
    # Build table rows
    table_rows = ""
    for result in results:
        row = '<tr class="results-data-row">'
        row += f'<td class="col-rank">{result["rank"]}</td>'
        row += f'<td class="col-chest">{escape_html(result.get("chestNumber", "-") or "-")}</td>'
        row += f'<td class="col-team">{escape_html(result.get("teamName", "-") or "-")}</td>'
        row += f'<td class="col-school">{escape_html(result.get("schoolName", "-") or "-")}</td>'
        
        # Add judge scores
        for judge_score in result.get("judgeScores", []):
            score = judge_score.get("score")
            if score is not None:
                if score > 10:
                    score = score / 10.0
                row += f'<td class="col-judge">{score:.2f}</td>'
            else:
                row += '<td class="col-judge">-</td>'
        
        # Add total score, grade score, grade, points
        total_score = result.get('totalScore', 0)
        if total_score > 100:
            total_score = total_score / 10.0
        
        normalized_score = result.get('normalizedScore', 0)
        row += f'<td class="col-total">{total_score:.2f}</td>'
        row += f'<td class="col-grade-score">{normalized_score:.2f}</td>'
        row += f'<td class="col-grade">{escape_html(result.get("grade", "-"))}</td>'
        row += f'<td class="col-points">{result.get("gradePoint", 0)}</td>'
        row += '</tr>'
        table_rows += row
    
    # Build judges signature blocks (outside table, on left side) - using table cells
    judges_signature_html = ""
    for i, judge in enumerate(judges, 1):
        judge_name = escape_html(judge["judgeName"])
        # Last judge has no right padding
        padding_style = "padding: 0;" if i == len(judges) else "padding: 0 8mm 0 0;"
        judges_signature_html += f'''
            <td style="border: none; {padding_style} vertical-align: bottom;">
                <div class="signature-block">
                    <div class="signature-line"></div>
                    <div class="signature-name">{judge_name}</div>
                    <div class="signature-label">Judge - {i}</div>
                </div>
            </td>
        '''
    
    # Replace template variables
    html_content = template.replace('{{total_columns}}', str(total_columns))
    html_content = html_content.replace('{{event_info_colspan}}', str(event_info_colspan))
    html_content = html_content.replace('{{empty_span}}', str(empty_span))
    html_content = html_content.replace('{{type_info_colspan}}', str(type_info_colspan))
    html_content = html_content.replace('{{category_info_colspan}}', str(category_info_colspan))
    html_content = html_content.replace('{{event_name}}', escape_html(event.get("name", "")))
    html_content = html_content.replace('{{category}}', escape_html(event.get("ageCategory", "")))
    html_content = html_content.replace('{{event_type}}', escape_html(event.get("eventType", "")))
    html_content = html_content.replace('{{judge_headers}}', judge_headers)
    html_content = html_content.replace('{{table_rows}}', table_rows)
    html_content = html_content.replace('{{judges_signature_html}}', judges_signature_html)
    
    # Create a subfolder for this report (without .pdf extension)
    report_folder_name = output_filename.replace('.pdf', '')
    report_folder_path = os.path.join(OUTPUT_DIR, report_folder_name)
    os.makedirs(report_folder_path, exist_ok=True)
    
    # Set output paths
    pdf_path = os.path.join(report_folder_path, output_filename)
    html_path = os.path.join(report_folder_path, output_filename.replace('.pdf', '.html'))
    
    # Save HTML file
    with open(html_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    # Generate PDF from HTML (base_url points to assets for font loading)
    HTML(string=html_content, base_url=Path(ASSETS_DIR).absolute()).write_pdf(pdf_path)
    
    print(f"✓ PDF generated: {pdf_path}")
    print(f"✓ HTML generated: {html_path}")

print("PDF generation function defined")

In [ ]:
# Generate report for one event
# Prompt for event ID when running (no need to edit code)
EVENT_ID = input("Enter event ID: ").strip()
if not EVENT_ID:
    raise ValueError("Event ID is required. Please run the cell again and enter an event ID.")

print(f"Fetching results for event ID: {EVENT_ID}")
try:
    event_data = fetch_event_results(EVENT_ID)
    
    # Generate filename (without path, will be saved in output folder)
    event_name = event_data["event"]["name"].replace(" ", "_").replace("/", "_")
    category = event_data["event"]["ageCategory"].replace(" ", "_")
    output_filename = f"report_{event_name}_{category}.pdf"
    
    # Create PDF and HTML
    create_report_pdf(event_data, output_filename)
    
    # Report folder path
    report_folder_name = output_filename.replace('.pdf', '')
    report_folder_path = os.path.join(OUTPUT_DIR, report_folder_name)
    
    print(f"\n✓ Report generated successfully!")
    print(f"  Event: {event_data['event']['name']}")
    print(f"  Category: {event_data['event']['ageCategory']}")
    print(f"  Results: {len(event_data['results'])} registration(s)")
    print(f"  Output folder: {report_folder_path}")
    print(f"    - PDF: {output_filename}")
    print(f"    - HTML: {output_filename.replace('.pdf', '.html')}")
except Exception as e:
    print(f"Error generating report: {e}")
    import traceback
    traceback.print_exc()